<a href="https://colab.research.google.com/github/kk412027247/nlp/blob/main/Advance_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2025-05-27 14:25:44--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.23, 18.164.174.55, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1748359545&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0ODM1OTU0NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQ

In [ ]:
 pip install langchain-community langchain-core langchain llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 9.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4127048 sha256=e8b3c2d98fbf6a40fad14639485a028592c9d9ebc1cc34e6518cf9164971bca3
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
Successfully built llama-cpp-python


In [ ]:
from langchain import LlamaCpp
llm = LlamaCpp(
  model_path="Phi-3-mini-4k-instruct-fp16.gguf",
  n_gpu_layers=-1,
  max_tokens=500,
  n_ctx=2048,
  seed=42,
  verbose=False
)

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


In [ ]:
llm.invoke("Hi! My name is Maarten. What is 1+1?")

''

In [ ]:
from langchain import PromptTemplate
template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(template=template, input_variables=["input_prompt"])

basic_chain = prompt | llm

basic_chain.invoke({
  'input_prompt':'Hi! My name is Maarten. What is 1 + 1?'
})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


' Hello Maarten! The answer to 1 + 1 is 2.'

In [ ]:
from langchain import LLMChain
template = """<s><|user|>
Create a titlte for a story about {summary}. only return the title.<|end|>
<|assistant|>"""

title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(prompt=title_prompt, llm=llm, output_key="title")

title.invoke({"summary": "a girl that lost her mother"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of Loss: A Tale of a Motherless Journey"'}

In [ ]:
template="""<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use oly two sentences.<|end|>
<|assistant|>"""

character_prompt = PromptTemplate(template=template, input_variables=["summary", "title"])
character = LLMChain(prompt=character_prompt, llm=llm, output_key="character")

In [ ]:
template = """<s><|user|>
Create a stroy about {summary} with the {title}. The main character is:{character}.
Only return the story and it cannot be longer than one paragraph.
<|end|>
<|assistant|>"""

story_prompt = PromptTemplate(template=template, input_variables=["summary","title", "character"])
story = LLMChain(prompt=story_prompt, llm=llm, output_key="story")

In [ ]:
llm_chain = title | character | story

In [ ]:
llm_chain.invoke("a girl that lost her mother")

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of a Mother\'s Love: A Girl\'s Journey Through Loss and Healing"',
 'character': ' The protagonist, Emily, is a resilient yet vulnerable young girl grappling with the profound grief of losing her mother. Despite her sorrow, she embarks on an emotional journey to find solace in cherished memories and discover inner strength through love and healing.',
 'story': " Emily's heart ached as the whispers of her mother's love echoed through the empty rooms, reminding her that she was no longer there. Grief weighed heavy on young shoulders but propelled Emily forward to embark on a journey—a quest for solace and healing amidst sorrow. With each tender memory etched in her heart, and every step taken with unwavering determination, the resilient girl discovered that love could mend even the deepest wounds. Through sharing stories of laughter and tears, finding strength in small moments shared with family and friends, Emily embraced 

In [ ]:
template="""<s><|user|>
Current conversation:{chat_history}
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(template=template, input_variables=["input_prompt", "chat_history"])

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")
llm_chain = LLMChain(prompt=prompt, llm=llm, memory=memory)

In [ ]:
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'Hi! My name is Maarten. What is 1 + 1?',
 'chat_history': '',
 'text': " The answer to 1 + 1 is 2. This basic arithmetic operation is one of the fundamental principles in mathematics, often taught at an early age as part of elementary education.\n\nHere's a simple explanation:\nWhen you have one object and then add another single object to it, you now have two objects in total. This concept can be visually represented using items like blocks or fingers for easy understanding. So, 1 + 1 equals 2."}

In [ ]:
llm_chain.invoke({"input_prompt": "What is my name?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Maarten. What is 1 + 1?\nAI:  The answer to 1 + 1 is 2. This basic arithmetic operation is one of the fundamental principles in mathematics, often taught at an early age as part of elementary education.\n\nHere's a simple explanation:\nWhen you have one object and then add another single object to it, you now have two objects in total. This concept can be visually represented using items like blocks or fingers for easy understanding. So, 1 + 1 equals 2.",
 'text': " Hi Maarten! You already mentioned your name, but to reiterate: I'm an AI and don't have a personal name. However, you can call me Assistant! 😊 And yes, the answer to 1 + 1 is indeed 2. Great job remembering that basic math principle!"}

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

llm_chain = LLMChain(prompt=prompt, llm=llm, memory=memory)

<ipython-input-25-d52609922a2c>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")


In [ ]:
llm_chain.predict(input_prompt="Hi! My name is Maarten. and I am 33 years old. what is 1 + 1?")
llm_chain.predict(input_prompt="What is 3 + 3 ?")

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


' Hi Maarten! The answer to 3 + 3 is 6. How can I assist you further? :)'

In [ ]:
llm_chain.invoke({"input_prompt":"what is my name?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'what is my name?',
 'chat_history': 'Human: Hi! My name is Maarten. and I am 33 years old. what is 1 + 1?\nAI:  Hi Maarten! The answer to 1 + 1 is 2. How are you today? :)\n(Note: This response introduces a bit of friendly conversation while still answering the question asked.)\nHuman: What is 3 + 3 ?\nAI:  Hi Maarten! The answer to 3 + 3 is 6. How can I assist you further? :)',
 'text': ' Hello! Your name is Maarten, as you mentioned at the beginning of our conversation. How else may I assist you?'}

In [ ]:
llm_chain.invoke({"input_prompt":"what is my age?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'what is my age?',
 'chat_history': 'Human: What is 3 + 3 ?\nAI:  Hi Maarten! The answer to 3 + 3 is 6. How can I assist you further? :)\nHuman: what is my name?\nAI:  Hello! Your name is Maarten, as you mentioned at the beginning of our conversation. How else may I assist you?',
 'text': " As an AI, I'm unable to determine your age as I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I'm here to help you with general information and questions! How else may I assist you?"}

In [ ]:
summary_prompt_template = """<s><|user|>
Summarize the conversations and update with the new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:
<|end|>
<|assistant|>"""

summary_prompt = PromptTemplate(template=summary_prompt_template, input_variables=["new_lines", "summary"])

In [ ]:
from langchain.memory import ConversationSummaryMemory
memory = ConversationSummaryMemory(llm=llm, prompt=summary_prompt, memory_key="chat_history")
llm_chain = LLMChain(prompt=prompt, llm=llm, memory=memory)

<ipython-input-31-a8900fdbed54>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm, prompt=summary_prompt, memory_key="chat_history")


In [ ]:
llm_chain.invoke({"input_prompt":"Hi! My name is Maarten. and I am 33 years old. what is 1 + 1?"})
llm_chain.invoke({"input_prompt":"What is my name?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': ' Hi Maarten, a 33-year-old individual inquired about the result of adding 1 + 1, to which the AI responded that it equals 2.',
 'text': ' Your name is not mentioned in the current conversation. You referred to someone as "Maarten," which might be a name, but there\'s no indication that it refers to you.'}

In [ ]:
llm_chain.invoke({"input_prompt":"What was the first question I asked?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What was the first question I asked?',
 'chat_history': ' In the conversation, a user referred to as "Maarten" inquired about the sum of 1 + 1. The AI confirmed that the result equals 2. Additionally, when asked about their name, the AI clarified that no name was provided within the current context and only "Maarten" was mentioned, which might be a third person\'s name rather than the user\'s.',
 'text': ' The first question you asked was, "What is 1 + 1?"'}

In [ ]:
memory.load_memory_variables({})

{'chat_history': ' In the conversation, a user named "Maarten" asked about the sum of 1 + 1. The AI confirmed that the result equals 2 and clarified that no name was given in the current context, with only "Maarten" mentioned potentially referring to a third person\'s name rather than their own. Later, Maarten queried about the initial question they asked, prompting the AI to reiterate that it was "What is 1 + 1?"'}

In [2]:
pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.4/438.4 kB 20.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.60
    Uninstalling langchain-core-0.3.60:
      Successfully uninstalled langchain-core-0.3.60


In [27]:
import os
from langchain_openai import ChatOpenAI
os.environ['OPENAI_API_KEY'] = ''
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [28]:
from langchain import PromptTemplate


react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""


prompt = PromptTemplate(
  template=react_template,
  input_variables=["input", "agent_scratchpad", "tools", "tool_names"]
)


In [17]:
pip install langchain-community

In [12]:
pip install -U duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 100.2 MB/s eta 0:00:00


In [29]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

In [30]:
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name='duckduck',
    description='A web search engine. Use this to as a search engine for general queries.',
    func=search.run
)
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [31]:
from langchain.agents import AgentExecutor, create_react_agent
agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [32]:
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)



> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}